In [25]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

In [35]:
filepath = "Resources/SaoPauloBrazilAQI.csv"
SaoAQIDF = pd.read_csv(filepath , delimiter=",")
SaoAQIDF

,date,pm25,pm10,o3,no2,so2,co
0,1/1/2014,,,17,6,,4
1,1/2/2014,43,,24,13,,5
2,1/3/2014,32,,26,16,,5
3,1/4/2014,41,,25,10,,4
4,1/5/2014,38,,25,8,,4
...,...,...,...,...,...,...,...
2361,7/20/2020,64,43,20,18,,9
2362,7/21/2020,70,51,18,23,,11
2363,7/22/2020,84,59,33,26,,11
2364,7/23/2020,102,46,35,26,,8


In [37]:
SaoAQIDF["date"] = SaoAQIDF["date"].astype(str)
SaoAQIDF[["month","day","year"]] = SaoAQIDF["date"].str.split("/", n=2, expand=True)
SaoAQIDF

ValueError: could not convert string to float: 

In [14]:
SaoAQIDF["month"] = SaoAQIDF["month"].astype(int)
SaoAQIDF["year"] = SaoAQIDF["year"].astype(int)
SaoAQIDF = SaoAQIDF[SaoAQIDF["month"] < 8]
SaoAQIDF["month"] = SaoAQIDF["month"].astype(str)
SaoAQIDF

,date,pm25,pm10,o3,no2,so2,co,month,day,year
0,1/1/2014,,,17,6,,4,1,1,2014
1,1/2/2014,43,,24,13,,5,1,2,2014
2,1/3/2014,32,,26,16,,5,1,3,2014
3,1/4/2014,41,,25,10,,4,1,4,2014
4,1/5/2014,38,,25,8,,4,1,5,2014
...,...,...,...,...,...,...,...,...,...,...
2361,7/20/2020,64,43,20,18,,9,7,20,2020
2362,7/21/2020,70,51,18,23,,11,7,21,2020
2363,7/22/2020,84,59,33,26,,11,7,22,2020
2364,7/23/2020,102,46,35,26,,8,7,23,2020


In [15]:
Sao2017AQIDF = SaoAQIDF[SaoAQIDF["year"] == 2017]
Sao2017AQIDF

,date,pm25,pm10,o3,no2,so2,co,month,day,year
1079,1/1/2017,,,29,2,,5,1,1,2017
1080,1/2/2017,,,2,,,,1,2,2017
1081,1/3/2017,,21,54,14,,8,1,3,2017
1082,1/4/2017,54,18,41,16,,6,1,4,2017
1083,1/5/2017,44,19,29,20,,3,1,5,2017
...,...,...,...,...,...,...,...,...,...,...
1286,7/27/2017,,,22,24,,5,7,27,2017
1287,7/28/2017,,,18,28,,4,7,28,2017
1288,7/29/2017,,,22,23,,4,7,29,2017
1289,7/30/2017,,,23,12,,3,7,30,2017


In [32]:
Sao2017AQIDF[" pm25"].count

<bound method Series.count of 0        
1        
2        
3      54
4      44
       ..
207      
208      
209      
210      
211      
Name:  pm25, Length: 212, dtype: object>

In [21]:
Sao2017AQIDF.dropna(subset=[" pm25"], inplace=True)
Sao2017AQIDF.drop(columns=[" pm10", " o3", " no2"," so2"," co","date","year","day"], inplace=True)
Sao2017AQIDF.reset_index(drop=True, inplace=True)
AQI2017 = []

for value in Sao2017AQIDF[" pm25"]:
    if value > 0 and value <= 12:
        AQI2017.append((((50-0)/(12 - 0))*(value - 0)) + 0)
    elif value >= 12.1 and value <= 35.4:
        AQI2017.append((((100 - 51)/(35.4 - 12.1))*(value - 12.1)) + 51)
    elif value >= 35.5 and value <= 55.4:
        AQI2017.append((((150 - 101)/(55.4 - 35.5))*(value - 35.5)) + 101)
    elif value >= 55.5 and value <= 150.4:
        AQI2017.append((((200 - 151)/(150.4- 55.4))*(value - 55.5)) + 151)
    elif value >= 150.5 and value <= 250.4:
        AQI2017.append((((300 - 201)/(250.4 - 150.5))*(value - 150.5)) + 201)
    elif value >= 250.5 and value <= 350.4:
        AQI2017.append((((400 - 301)/(350.4 - 250.5))*(value - 250.5)) + 301)
    elif value >= 350.5 and value <= 500.4:
        AQI2017.append((((500 - 401)/(500.4 - 350.5))*(value - 350.5)) + 401)
    else:
        AQI2017.append("invalid")
#Add list to data frame, then drop rows with invalid values
Sao2017AQIDF["AQI"] = AQI2017
Sao2017AQIDF = Sao2017AQIDF[Sao2017AQIDF["AQI"] != "invalid" ]
#round it up
Sao2017AQIDF["AQI"] = [round(i) for i in Sao2017AQIDF["AQI"]]

Sao2017 = Sao2017AQIDF.groupby(["month"]).mean()
Sao2017.rename(columns={"AQI":"2017 Mean AQI"},index={"1":"Jan", "2":"Feb", "3":"March", "4":"April", "5":"May", "6":"June", "7":"July"}, inplace=True)
Sao2017.drop([" pm25"], axis=1, inplace=True)
Sao2017 = Sao2017.astype(int)
Sao2017

KeyError: "[' pm10' ' o3' ' no2' ' so2' ' co' 'date' 'year' 'day'] not found in axis"